In [ ]:
######################## Import libraries and drivers ########
from pynq import Overlay      # Used to load FPGA Overay
from pynq.drivers import DMA  # Used to interface with DMA blocls
from time import sleep        # Pause CPU
import numpy as np            # Used for window function
import serial                 # Serial Communication
import Setup                  # Set up hardware blocks
import data_read              # Read hardware block outputs
import note_detect            # Detect notes played
########################### Overlay ###########################
# Overlay bitstream location
ol = Overlay("RTAA.bit")
# Download bitstream to kernel
ol.download()
########################### Constants #########################
Threshold = 1800
offset = 1720                # Voltage offset at input
transfer_XADC = 8192         # transfer size of ADC DMA block
transfer_XFFT = 8192         # transfer size of FFT DMA block
n = '0000000000000000000000000000000000000' # initialise Note variable
######### FFT setup scaling factors and fwd/inverse ##########
# /4 /2 /4 /2 /4 /2 /4 /2 /4 /2 /4 /2 /4 /2    FWD(1)/INV(0)
set_FFT = 0b000000000010101  # 
########################### Kaiser Window ####################
w = np.kaiser(2048, 2);
########################## IP Setup ##########################
Setup.gpio_setup(transfer_XADC, transfer_XFFT, set_FFT)
########################## DMA Setup #########################
dma_xadc= DMA(0x40420000, 1)     # Object for XADC DMA block
dma_ffti = DMA(0x40410000, 0)    # Object for FFT input DMA block
dma_ffto= DMA(0x40400000, 1)     # Object for FFT output DMA block
dma_ffti.create_buf(8192)
ffti_buffer = dma_ffti.get_buf(32) # FFT input buffer for DMA
dma_xadc.create_buf(8192)
xadc_buffer = dma_xadc.get_buf(32) # ADC output buffer for DMA
dma_ffto.create_buf(8192)
ffto_buffer = dma_ffto.get_buf(32) # FFT output buffer for DMA
####################### Serial Connection Setup ##############
ser = serial.Serial(port='/dev/ttyPS0',baudrate=115200)
###################### Main Loop #############################
while(1):
    ################# FFT Input ##############################
    dma_xadc.transfer(transfer_XADC, 1) # transfer ADC values from FPGA
    sleep(0.05)
    for i in range(2048):               # fill input buffer with windowed ADC output values
        ffti_buffer[i] = int(w[i]*(data_read.xadc_val(xadc_buffer[i])-offset) + offset)   
    dma_ffti.transfer(transfer_XFFT, 0) # transfer first 2048 values to fft block
    for i in range(2048):               # append the input with the voltage offset
        ffti_buffer[i] = offset
    for i in range(3):                 # transfer the remaining 6144 values
        dma_ffti.transfer(transfer_XFFT, 0)
    ################# FFT Output #############################
    dma_ffto.transfer(transfer_XFFT, 1) # transfer the output of the fft from the FPGA
    f4 = [0]*512                        # k vales for fft ouput where resolution is 4 Hz
    for i in range(0,2048):
        if(data_read.comp(ffto_buffer[i])>(Threshold)):            # threshold to determine note is present
            f4[i]=1           # set element to high if note is present
    ############### Note Detection ##########################
    n = note_detect.note(f4)
    ser.write(str.encode(n))      # Send a string to a PC via serial

    for i in range(3):
        dma_ffto.transfer(transfer_XFFT, 1)
        dma_ffto.free_buf()
        dma_ffto.create_buf(8192)
        ffto_buffer = dma_ffto.get_buf(32)
    
ser.close()             # close serial port